<a href="https://colab.research.google.com/github/sinjy1203/deep-learning/blob/master/mnist_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [0]:
x_train1 = x_train.reshape(-1, 28, 28, 1).astype(np.float32)
y_train1 = y_train.astype(np.int32)
x_test1 = x_test.reshape(-1, 28, 28, 1).astype(np.float32)
y_test1 = y_test.astype(np.int32)
from sklearn.model_selection import train_test_split
x_train2, x_val = train_test_split(x_train1, test_size=0.2, random_state=42)
y_train2, y_val = train_test_split(y_train1, test_size=0.2, random_state=42)

In [4]:
import time
tf.reset_default_graph()
tf.set_random_seed(42)
np.random.seed(42)

def shuffle_batch(x, y, batch_size):
  rnd_idx = np.random.permutation(len(x))
  n_batch = len(x) // batch_size
  for batch_idx in np.array_split(rnd_idx, n_batch):
    xs, ys = x[batch_idx], y[batch_idx]
    yield xs, ys

start = time.time()
x = tf.placeholder(dtype=tf.float32, shape=(None, 28, 28, 1), name='x')
y = tf.placeholder(dtype=tf.int32, shape=(None), name='y')
training = tf.placeholder_with_default(False, shape=[])
he_init = tf.variance_scaling_initializer()
conv1 = tf.layers.conv2d(x, filters=10, kernel_size=3, strides=[1,1], activation=tf.nn.elu, padding='SAME')
max_pool1 = tf.nn.max_pool(conv1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
conv2 = tf.layers.conv2d(max_pool1, filters=50, kernel_size=3, strides=(1,1), activation=tf.nn.elu, padding='SAME')
max_pool2 = tf.nn.max_pool(conv2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

x_reshape = tf.reshape(max_pool2, shape=(-1, 7*7*50))
layer = x_reshape
n_layers = 3


for i in range(n_layers):
  if i == n_layers - 1:
    layer = tf.layers.dropout(layer, rate=0.5, training=training)
    layer = tf.layers.dense(layer, 10, kernel_initializer=he_init)
  else:
    layer = tf.layers.dropout(layer, rate=0.5, training=training)
    layer = tf.layers.dense(layer, 100, kernel_initializer=he_init)
    layer = tf.layers.batch_normalization(layer, momentum=0.9, training=training)
    layer = tf.nn.elu(layer)
  
  # layer = tf.layers.batch_normalization(layer, momentum=0.9, training=training, )

logits = layer
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits, name='xentropy')
loss = tf.reduce_mean(xentropy)
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
training_op = optimizer.minimize(loss)

correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
init = tf.global_variables_initializer()

# file_writer = tf.summary.FileWriter("./log", tf.get_default_graph())
with tf.Session() as sess:
  init.run()
  for epoch in range(51):
    for xs, ys in shuffle_batch(x_train2, y_train2, 100):
      _, cost_val, __ = sess.run([training_op, loss, extra_update_ops], feed_dict={x: xs, y: ys, training: True})
    if epoch % 10 == 0:
      accuracy_val = accuracy.eval(feed_dict={x: x_val, y: y_val})
      print(accuracy_val, cost_val)
  accuracy_test = accuracy.eval(feed_dict={x: x_test1, y: y_test1})
  print("정확도: ", accuracy_test) 
print("걸린 시간: ", time.time() - start)####정확도 99.3%

Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).
0.98 0.15386526
0.99116665 0.06706425
0.99175 0.023364717
0.99258333 0.04016701
0.99325 0.018633695
0.99275 0.0035382395
정확도:  0.9933
걸린 시간:  135.43223524093628
